# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory

import pandas as pd
from sklearn.model_selection import train_test_split

from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_disease_automl'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [4]:
cpu_cluster_name = "capstone-comp"
# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [5]:
URL = "https://raw.githubusercontent.com/guptaa98/Microsoft_Azure_Nanodegree_Projects/main/Captstone%20Project/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=URL)

In [6]:
x = ds.to_pandas_dataframe().dropna()
y = x.pop('target')

x_train, x_test, y_train, y_test = train_test_split(x, y)
dataset = pd.concat([x_train,y_train],axis=1)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(experiment_timeout_minutes=20,
                             task="classification",
                             primary_metric="accuracy",
                             training_data=dataset,
                             label_column_name='target',
                             n_cross_validations=5)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
import joblib
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print(best_run)
print(best_run_metrics['accuracy'])
print(fitted_model)

Run(Experiment: heart_disease_automl,
Id: AutoML_3fa58aab-ed3e-4b9c-9da8-ce8090e48a0b_41,
Type: None,
Status: Completed)
0.8721739130434782
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_samples=None,
                                                                                                    min_impurity_decrease=0.0,
       

In [23]:
#TODO: Save the best model

joblib.dump(fitted_model, "best_automl_model.pkl")

['best_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [31]:
from azureml.core.model import Model
model = Model.register(model_path="best_automl_model.pkl",
                       model_name="best_automl_model",
                       workspace = ws)

Registering model best_automl_model


In [32]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/10b5a759/classes.interpret.json',
 'explanation/10b5a759/eval_data_viz.interpret.json',
 'explanation/10b5a759/expected_values.interpret.json',
 'explanation/10b5a759/features.interpret.json',
 'explanation/10b5a759/global_names/0.interpret.json',
 'explanation/10b5a759/global_rank/0.interpret.json',
 'explanation/10b5a759/global_values/0.interpret.json',
 'explanation/10b5a759/local_importance_values.interpret.json',
 'explanation/10b5a759/per_class_names/0.interpret.json',
 'explanation/10b5a759/per_class_rank/0.interpret.json',
 'explanation/10b5a759/per_class_values/0.interpret.json',
 'explanation/10b5a759/rich_metadata.interpret.json',
 'explanation/10b5a759/visualization_dict.interpret.json',
 'explanation/a0e1ee04/classes.interpret.json',
 'explanation/a0e1ee04/expected_values.interpret.json',
 'explanation/a0e1ee04/features.interpret.json',
 'explanation/a0e1ee04/global_names/0.interpret.json',
 'explanation/a0e1ee04/global_rank/0.interpret.json',
 'explanation/a0e1ee04/global_values/0.interpret.json',
 'explanation/a0e1ee04/local_importance_values.interpret.json',
 'explanation/a0e1ee04/per_class_names/0.interpret.json',
 'explanation/a0e1ee04/per_class_rank/0.interpret.json',
 'explanation/a0e1ee04/per_class_values/0.interpret.json',
 'explanation/a0e1ee04/rich_metadata.interpret.json',
 'explanation/a0e1ee04/visualization_dict.interpret.json',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [33]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [34]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'myenv.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service